In [24]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib.pyplot as plt

import math
import timeit
import os

import tensorflow as tf
import keras
from keras import backend as K

import cv2
import scipy
import skimage

%matplotlib inline

#Some tensorflow gpu stuff

K.tensorflow_backend._get_available_gpus()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13547564351894836280
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1508653533131984402
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9878259401148055522
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14450614160694504218
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]
Num GPUs Available:  1


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### GAN Model

In [0]:
#Deconvolutional NN: Conv2DTranspose reverses the process of a standard CNN. Batch Norm to make it easier in training.

class Generator():
    
    def __init__(self):        
        self.layers = keras.models.Sequential()
        self.layers.add(keras.layers.Reshape((1,1,100)))
        self.layers.add(keras.layers.Dense(units=1024 * 4 * 4, input_dim=100, activation='linear'))
        self.layers.add(keras.layers.Conv2DTranspose(filters=int(1024), kernel_size=12))
        self.layers.add(keras.layers.Conv2DTranspose(filters=int(1024/2), kernel_size=16, activation='relu'))
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Conv2DTranspose(filters=int(1024/4), kernel_size=16, activation='relu'))
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Conv2DTranspose(filters=int(1024/8), kernel_size=16, activation='relu'))        
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Conv2DTranspose(filters=int(1024/16), kernel_size=16, activation='relu'))
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Conv2DTranspose(filters=int(1024/32), kernel_size=16, activation='relu'))
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Conv2DTranspose(filters=3, kernel_size=10, activation='tanh'))
        self.layers.add(keras.layers.BatchNormalization())
        
        self.inputs = keras.layers.Input((100,)) #input the z space
        
        self.outputs = self.layers(self.inputs)
                
        self.model = keras.models.Model(inputs=self.inputs, outputs=self.outputs) #output a NN
        

In [0]:
#Convolutional NN: Standard classification NN (no particular model like resNet or VGG-16, just standard DCNN for GANs.)

class Discriminator():
    
    def __init__(self):        
        self.layers = keras.models.Sequential()
        
        self.layers.add(keras.layers.Conv2D(filters=64, kernel_size=4))
        self.layers.add(keras.layers.LeakyReLU(0.2))
        self.layers.add(keras.layers.Dropout(0.3))
        self.layers.add(keras.layers.Conv2D(filters=64*2, kernel_size=4))
        self.layers.add(keras.layers.LeakyReLU(0.2))
        self.layers.add(keras.layers.Dropout(0.3))
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Conv2D(filters=64*4, kernel_size=4))
        self.layers.add(keras.layers.LeakyReLU(0.2))
        self.layers.add(keras.layers.Dropout(0.3))
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Conv2D(filters=64*8, kernel_size=4))
        self.layers.add(keras.layers.LeakyReLU(0.2))
        self.layers.add(keras.layers.Dropout(0.3))
        self.layers.add(keras.layers.BatchNormalization())
        self.layers.add(keras.layers.Dense(1024 * 4 * 4, input_dim=1, activation='linear'))

        self.layers.add(keras.layers.Flatten())    
        
        self.inputs = keras.layers.Input((96, 96, 3)) #input an image
        
        features = self.layers(self.inputs)
        
        self.outputs = keras.layers.Dense(1, activation='sigmoid')(features)
        
        self.model = keras.models.Model(inputs=self.inputs, outputs=self.outputs) #output a NN
        

In [28]:
discriminator = Discriminator()
generator = Generator()

#Compilation of loss, metrics, optimizers
discriminator.model.compile(optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5),
                     loss=['binary_crossentropy'],
                     metrics=['binary_accuracy'],
                           options=run_opts)
discriminator.model.summary()

generator.model.compile(optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5),
                     loss=['binary_crossentropy'],
                     metrics=['binary_accuracy'],
                           options=run_opts)
generator.model.summary()

#Uncomment one by one to try different loss rate/compilations
'''
discriminator.model.compile(optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5),
                     loss=['sparse_categorical_crossentropy'],
                     metrics=['sparse_categorical_crossentropy'])
discriminator.model.summary()

generator.model.compile(optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5),
                     loss=['sparse_categorical_crossentropy'],
                     metrics=['sparse_categorical_crossentropy'])
generator.model.summary()
'''

'''
discriminator.model.compile(optimizer='adagrad',
                     loss=['binary_crossentropy'],
                     metrics=['binary_accuracy'])
discriminator.model.summary()

generator.model.compile(optimizer='adagrad',
                     loss=['binary_crossentropy'],
                     metrics=['binary_accuracy'])
generator.model.summary()
'''

'''
discriminator.model.compile(optimizer='adagrad',
                     loss=['sparse_categorical_crossentropy'],
                     metrics=['sparse_categorical_crossentropy'])
discriminator.model.summary()

generator.model.compile(optimizer='adagrad',
                     loss=['sparse_categorical_crossentropy'],
                     metrics=['sparse_categorical_crossentropy'])
generator.model.summary()
'''

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
sequential_9 (Sequential)    (None, 115605504)         11165120  
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 115605505 
Total params: 126,770,625
Trainable params: 126,768,833
Non-trainable params: 1,792
_________________________________________________________________
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 100)               0         
_________________________________________________________________
sequential_10 (Sequential)   (None, 96, 96, 3)         2596371695
Total params: 2,596,371

"\ndiscriminator.model.compile(optimizer='adagrad',\n                     loss=['sparse_categorical_crossentropy'],\n                     metrics=['sparse_categorical_crossentropy'])\ndiscriminator.model.summary()\n\ngenerator.model.compile(optimizer='adagrad',\n                     loss=['sparse_categorical_crossentropy'],\n                     metrics=['sparse_categorical_crossentropy'])\ngenerator.model.summary()\n"

### Training/Testing Process (copied from previous GAN.py)

In [30]:
netD_real_input = keras.layers.Input(shape=(96, 96, 3))
noisev = keras.layers.Input(shape=(100,))

loss_real = K.mean(discriminator.model(netD_real_input))
loss_fake = K.mean(discriminator.model(generator.model(noisev)))
loss = loss_fake - loss_real 
training_updates = keras.optimizers.Adam(lr=0.0002, beta_1=0.5).get_updates(discriminator.model.trainable_weights,[], loss)
netD_train = K.function([netD_real_input, noisev],
                        [loss_real, loss_fake],    
                        training_updates)

loss = -loss_fake 
training_updates2 = keras.optimizers.Adam(lr=0.0002, beta_1=0.5).get_updates(generator.model.trainable_weights,[], loss)
netG_train = K.function([noisev], [loss], training_updates2)

fixed_noise = np.random.normal(size=(64, 100)).astype('float32')

datagen = keras.preprocessing.image.ImageDataGenerator()

train_generate = datagen.flow_from_directory("/content/gdrive/My Drive/test/", target_size=(96,96), batch_size=4, 
                                                shuffle=True, class_mode=None)#, save_format='jpg')

Found 2 images belonging to 2 classes.


In [33]:
step = 0
#print(dir(train_generate))
for step in range(10):  
    
    for _ in range(5):
        real_data = (np.array(train_generate.next())*2-1)
        noise = np.random.normal(size=(64, 100))
        errD_real, errD_fake  = netD_train([real_data, noise])
        errD = errD_real - errD_fake
        netD_clamp([])

    noise = np.random.normal(size=(64, 100))  
    errG, = netG_train([noise])    
    print('[%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f' % (step, errD, errG, errD_real, errD_fake))
            
    if step%1==0:
        discriminator.model.save("discriminator_Adam.h5")
        generator.model.save("generate_Adam.h5")
        fake = generator.model.predict(fixed_noise)
        display_grid = np.zeros((8*96,8*96,3))
        
        for j in range(int(64/8)):
            for k in range(int(64/8)):
                display_grid[j*96:(j+1)*96,k*96:(k+1)*96,:] = fake[k+8*j]
        img_save_path = os.path.join(os.getcwd(),"saved/img_test/{}.png".format(step))
        plt.imsave(img_save_path, display_grid)
        


InternalError: ignored

# IGNORE BELOW

### Failed Training/Testing Process

In [0]:
directory = "data/augmented/test/"

count = 0
for folder in os.listdir(directory):
    if (folder != "_Fake"):
        for file in os.listdir(directory + folder + "/"):
            count += 1
            
for i in range(0, count):
    noise = np.random.normal(size=(96, 96)).astype('float32')
    saveFile = directory + "_Fake" + "/fake_" + str(i) + ".png"
    plt.imsave(saveFile, noise)

In [0]:
directory = "/content/gdrive/My Drive/test/"
datagen = keras.preprocessing.image.ImageDataGenerator()

#To do per class: rearrange the file structure to have 1 folder inside of another folder 
#in order for train_generate to find the images needed
#e.g. augmented -> test -> lava -> (images) / _Fake -> (images) so that there is 1 class named lava, one class named _Fake
#All images labelled as 1 (with image_generate.labels) in the lava folder
#All images labeled as 0 in the _Fake folder
image_generate = datagen.flow_from_directory(directory, target_size=(96,96), batch_size=1, shuffle=True, class_mode='binary')

discriminator.model.fit_generator(image_generate, steps_per_epoch=1, epochs=1)

#NOT ENOUGH COMPUTATION, ENOUGH ON GOOGLE COLAB!

Found 2 images belonging to 2 classes.



Epoch 1/1



ResourceExhaustedError: ignored

In [0]:
noise = np.random.normal(size=(64, 100)).astype('float32')
generator.model.fit(noise, steps_per_epoch=1, epochs=1)

In [0]:
#Number of iterations to predict
epoch = 1
fixed_noise = np.random.normal(size=(64, 100)).astype('float32')
for i in range(0, epoch):
    fake = generator.model.predict(fixed_noise)
    display_grid = np.zeros((8*18,8*18,3))
    
    for j in range(int(64/8)):
        for k in range(int(64/8)):
            display_grid[j*18:(j+1)*18,k*18:(k+1)*18,:] = fake[k+8*j]
            plt.imshow(fake[k+8*j])
            plt.show()

    #Change this save path
    img_save_path = os.path.join(os.getcwd(),"results_GAN/lava/" + str(i) + ".png")
    #plt.imsave(img_save_path, display_grid)

### Random Stuff, ignore (training/testing above)

In [0]:
'''
filename = "data/test folder/Tex_Tile_Solitude_Water-0.png"
I = plt.imread(filename)
        
#convert to float data type and scale to [0..1] if necessary
if (I.dtype != np.float32):
    I = I.astype(np.float32) / 256
I = cv2.resize(I, (96,96))
I = np.expand_dims(I, axis=0)

noise = np.random.normal(size=(96, 96)).astype('float32')
fake = generator.model.predict(noise)

discriminator.model.fit(I, [1], epochs=1, batch_size=64)
#generator.model.fit(noise, [0], epochs=1, batch_size=64)

display_grid = np.zeros((8*96,8*96,3))

for j in range(int(64/8)):
    for k in range(int(64/8)):
        display_grid[j*96:(j+1)*96,k*96:(k+1)*96,:] = fake[k+8*j]
        
if (display_grid.dtype != np.float32):
    display_grid = display_grid.astype(np.float32) / 256
display_grid[np.where(display_grid < 0)] = 0.0
display_grid[np.where(display_grid > 1)] = 1.0
plt.imsave("saved/results_GAN/brick/test.png", display_grid)
'''

In [0]:
#To mass train
for folder in os.listdir("data/augmented/"):
    for file in os.listdir("data/augmented/" + folder + "/"):
        filename = "data/augmented/" + folder + "/" + file
        I = plt.imread(filename)
        
        #convert to float data type and scale to [0..1] if necessary
        if (I.dtype != np.float32):
            I = I.astype(np.float32) / 256
        I = cv2.resize(I, (512,512))
        w, h, c = I.shape
        
        noise = np.random.normal(size=(64, 100)).astype('float32')
        
        discriminator.model.fit(I, epochs=10, batch_size=64)
        generator.model.fit(I, epochs=10, batch_size=64)
        
        break
        #Do more stuff, comment break statement

In [0]:
#To train one class
folder = "brick"
for file in os.listdir("data/augmented/" + folder + "/"):
    filename = "data/augmented/" + folder + "/" + file
    I = plt.imread(filename)
    
    #convert to float data type and scale to [0..1] if necessary
    if (I.dtype != np.float32):
        I = I.astype(np.float32) / 256
    I = cv2.resize(I, (512,512))
    w, h, c = I.shape
    
    noise = np.random.normal(size=(64, 100)).astype('float32')
    
    break
    #Do more stuff, comment break statement

In [0]:
#To store results
I = plt.imread("data/augmented/brick/bilateral_6_block_tiles_blue.png")
plt.imsave("saved/results_GAN/brick/test.png", I)